## Validacion Cruzada

In [1]:
# ============================================
# PINN Pozo Infinito 1D (TF 2.16+ / Keras 3)
# - Corridas simples, CV k-fold y comparaciones
# - Guardado centralizado en BASE_DIR (ResultadosV)
# - Mixed precision opcional (Tensor Cores)
# ============================================

import os, math, csv, time, random
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers, ops as K

# ---------- Ruta base de resultados ----------
BASE_DIR = "/home/david/Schr-dingerPINNsUQValidation/PrimeraFase/ValidacionPozoInfinito/ValidacionCruzadaVsPINNs/ResultadosV"
os.makedirs(BASE_DIR, exist_ok=True)

def path(*args):
    """
    Une rutas relativas dentro de BASE_DIR.
    Si la ruta apunta a un archivo, crea automáticamente el directorio padre.
    """
    p = os.path.join(BASE_DIR, *args)
    d = p if (os.path.splitext(p)[1] == "") else os.path.dirname(p)
    os.makedirs(d, exist_ok=True)
    return p

# --------- (Opcional) Silenciar logs y oneDNN en CPU ---------
# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"       # INFO/WARN off
# os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"      # evita mensajes oneDNN en CPU

# --------- (Recomendado) Habilitar memory-growth en GPU ---------
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except Exception:
        pass

# --------- (Opcional) Mixed precision para usar Tensor Cores ---------
USE_MIXED_PRECISION = True
if USE_MIXED_PRECISION:
    keras.mixed_precision.set_global_policy("mixed_float16")
print("Política actual:", keras.mixed_precision.global_policy())

# --------- Utils: integración trapezoidal (NumPy moderno) ----------
def trapz_np(y, x):
    # NumPy >= 1.26 recomienda trapezoid
    y = y.astype(np.float32, copy=False)
    return float(np.trapezoid(y, x=x))

# --------- Trial factor trigonométrico (Keras 3 safe) ------------
def trig_nodal_factor(x, n: int):
    """
    F(x) = sin(n*pi*x) / (sin(pi*x) + eps)
    x: KerasTensor (Input)
    n: entero (modo)
    """
    eps = K.cast(1e-12, x.dtype)
    s1  = K.sin(math.pi * x)
    sn  = K.sin(n * math.pi * x)
    ratio = sn / (s1 + eps)
    n_cast = K.cast(n, x.dtype)
    return K.where(K.abs(s1) < 1e-6, n_cast, ratio)

class Sine(layers.Layer):
    """Activación seno compatible con Keras 3 (usa keras.ops)."""
    def call(self, x):
        return K.sin(x)

def make_net(n=1, hidden=64, use_sine=True):
    """
    Red: psi(x) = x(1-x) * F_n(x) * N(x)
    - F_n(x) = sin(nπx)/sin(πx)
    - N(x): MLP (tanh o seno)
    """
    x_in = keras.Input(shape=(1,), dtype="float32")

    if use_sine:
        z = layers.Dense(hidden, activation=None,
                         kernel_initializer="glorot_uniform",
                         bias_initializer="zeros")(x_in)
        z = Sine()(z)
        z = layers.Dense(hidden, activation=None,
                         kernel_initializer="glorot_uniform",
                         bias_initializer="zeros")(z)
        z = Sine()(z)
    else:
        z = layers.Dense(hidden, activation="tanh",
                         kernel_initializer="glorot_uniform",
                         bias_initializer="zeros")(x_in)
        z = layers.Dense(hidden, activation="tanh",
                         kernel_initializer="glorot_uniform",
                         bias_initializer="zeros")(z)

    # IMPORTANTE en mixed precision: salida en float32 para la loss
    out = layers.Dense(1, activation=None,
                       kernel_initializer="glorot_uniform",
                       bias_initializer="zeros",
                       dtype="float32")(z)

    F = trig_nodal_factor(x_in, n)
    psi = K.multiply(x_in, (1.0 - x_in))
    psi = K.multiply(psi, F)
    psi = K.multiply(psi, out)
    return keras.Model(inputs=x_in, outputs=psi)

# --------- Derivadas de segundo orden con tapes ---------------
def second_derivative(model, x):
    x = tf.convert_to_tensor(x)
    x = tf.reshape(x, (-1,1))
    with tf.GradientTape(persistent=True) as t2:
        t2.watch(x)
        with tf.GradientTape() as t1:
            t1.watch(x)
            psi = model(x)
        psi_x = t1.gradient(psi, x)
    psi_xx = t2.gradient(psi_x, x)
    del t2
    return psi, psi_xx

# --------- Pérdida (PDE + normalización) ----------------------
@tf.function  # para más performance: @tf.function(jit_compile=True)
def compute_losses(net, x_batch, E, lam):
    psi, psi_xx = second_derivative(net, x_batch)
    res = psi_xx + E * psi                 # V(x)=0 en (0,1)
    LPDE = tf.reduce_mean(tf.square(res))
    psi2 = tf.squeeze(tf.square(psi), axis=1)
    xb   = tf.squeeze(tf.convert_to_tensor(x_batch), axis=1)
    dx   = xb[1:] - xb[:-1]
    integral = tf.reduce_sum(0.5*(psi2[1:] + psi2[:-1]) * dx)
    Lnorm = tf.square(integral - 1.0)
    L = LPDE + lam * Lnorm
    return L, LPDE, Lnorm, integral

# --------- Entrenamiento de un modo (corrida simple) ----------
def run_one_mode(n, save_subdir=None):
    if save_subdir is None:
        save_subdir = f"sin_cv_n{n}"

    # energía y hiperparámetros (heurística según n)
    E = np.float32((n * math.pi)**2)
    USE_SINE = True if n >= 3 else False
    HIDDEN   = 128 if n >= 3 else 64
    N_col    = max(1024, 2048*n)
    EPOCHS   = 15000 if n >= 4 else (9000 if n==3 else (6000 if n==2 else 4000))
    LR0      = 3e-4  if n >= 4 else (5e-4 if n==3 else (7e-4 if n==2 else 1e-3))

    lam_hi, lam_lo = (300.0, 80.0) if n >= 3 else (40.0, 15.0 if n==2 else 10.0)

    # red y puntos de colación
    net = make_net(n=n, hidden=HIDDEN, use_sine=USE_SINE)
    x_col = np.linspace(0,1,N_col, dtype=np.float32).reshape(-1,1)
    x_batch = tf.constant(x_col)

    # optimizador con decay + clipping
    lr_sched = keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=LR0, decay_steps=EPOCHS,
        end_learning_rate=LR0*0.1, power=1.0
    )
    opt = keras.optimizers.Adam(learning_rate=lr_sched, clipnorm=1.0)

    # historial
    loss_total, loss_pde, loss_norm = [], [], []

    # bucle de entrenamiento
    for ep in range(1, EPOCHS+1):
        lam = lam_hi if ep < EPOCHS//3 else lam_lo
        with tf.GradientTape() as tape:
            L, LPDE, Lnorm, integral = compute_losses(net, x_batch, E, lam)
        grads = tape.gradient(L, net.trainable_variables)
        opt.apply_gradients(zip(grads, net.trainable_variables))

        loss_total.append(float(L))
        loss_pde.append(float(LPDE))
        loss_norm.append(float(Lnorm))

        if ep % max(800, EPOCHS//6) == 0 or ep == 1:
            tf.print(f"n={n}", "ep", ep, "| LPDE=", LPDE, " Lnorm=", Lnorm,
                     " L=", L, " ∫|ψ|²≈", integral, " λ=", lam)

    # curva de pérdida
    plt.figure(figsize=(7,5))
    plt.semilogy(loss_total, label="Total Loss")
    plt.semilogy(loss_pde, label=r"$\mathcal{L}_{PDE}$")
    plt.semilogy(loss_norm, label=r"$\mathcal{L}_{norm}$")
    plt.xlabel("Épocas"); plt.ylabel("Pérdida"); plt.title(f"Curva de pérdida (n={n})")
    plt.legend()
    out_path_loss = path(save_subdir, f"loss_{n}.png")
    plt.savefig(out_path_loss, dpi=150, bbox_inches="tight"); plt.close()

    # evaluación fina
    xs = np.linspace(0,1,2000, dtype=np.float32).reshape(-1,1)
    psi_pred  = net(xs).numpy().squeeze().astype(np.float32)
    psi_exact = (np.sqrt(2.0)*np.sin(n*math.pi*xs)).squeeze().astype(np.float32)

    # alinear signo con robustez
    dot = float(np.dot(psi_pred, psi_exact))
    sign = np.sign(dot)
    if sign == 0:
        sign = 1.0
    psi_pred *= sign

    # métricas
    l2_err = float(np.sqrt(np.mean((psi_pred-psi_exact)**2)))
    integ  = trapz_np(psi_pred**2, x=xs.squeeze())

    # figura por modo
    plt.figure()
    plt.plot(xs.squeeze(), psi_pred, label=f"PINN ψ_{n}")
    plt.plot(xs.squeeze(), psi_exact, "--", label=f"Exacta ψ_{n}")
    plt.title(f"Modo n={n}  |  E={(n*math.pi)**2:.2f}  |  L2={l2_err:.2e}")
    plt.xlabel("x"); plt.ylabel("ψ"); plt.legend()
    out_path = path(save_subdir, f"modo_{n}.png")
    plt.savefig(out_path, dpi=150, bbox_inches="tight"); plt.close()

    return dict(n=n, xs=xs.squeeze(), psi_pred=psi_pred,
                psi_exact=psi_exact, E=float(E), L2=l2_err, integral=integ,
                fig_path=out_path, loss_path=out_path_loss,
                losses=loss_total, losses_pde=loss_pde, losses_norm=loss_norm)

# --------- Validación cruzada (k corridas) --------------------
def cross_validation_mode(n, k_folds=5):
    results = []
    print(f"\n=== VALIDACIÓN CRUZADA para modo n={n} con {k_folds} folds ===\n")
    for k in range(k_folds):
        # semillas consistentes por fold
        keras.utils.set_random_seed(k)
        np.random.seed(k)
        random.seed(k)

        res = run_one_mode(n, save_subdir=f"fold_{k+1}_n{n}")
        results.append(res)
        print(f"[fold {k+1}/{k_folds}] L2={res['L2']:.2e}  ∫|ψ|²≈{res['integral']:.3f}")

    L2s = np.array([r['L2'] for r in results], dtype=np.float32)
    integrals = np.array([r['integral'] for r in results], dtype=np.float32)

    print("\nPromedio y desviación estándar:")
    print(f"L2 promedio = {L2s.mean():.2e} ± {L2s.std():.2e}")
    print(f"∫|ψ|² promedio = {integrals.mean():.3f} ± {integrals.std():.3f}")

    # gráfico simple de barras de L2 por fold
    plt.figure(figsize=(7,5))
    plt.bar(range(1, k_folds+1), L2s)
    plt.xlabel("Fold"); plt.ylabel("Error L2"); plt.title(f"Validación cruzada (n={n})")
    plt.tight_layout()
    plt.savefig(path(f"validacion_n{n}.png"), dpi=150); plt.close()

    return results, L2s.mean(), L2s.std()

# --------- Comparador: SIN CV vs CON CV -----------------------
def single_run_mode(n, seed=0):
    keras.utils.set_random_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    return run_one_mode(n, save_subdir=f"sin_cv_n{n}")

def compare_single_vs_cv(n, k_folds=5, seed=0):
    base = single_run_mode(n, seed=seed)
    L2_base   = base['L2']
    norm_base = abs(base['integral'] - 1.0)

    results, mean_L2, std_L2 = cross_validation_mode(n, k_folds=k_folds)
    norm_errors = [abs(r['integral'] - 1.0) for r in results]
    mean_norm, std_norm = float(np.mean(norm_errors)), float(np.std(norm_errors))

    print(f"\n[RESUMEN n={n}]")
    print(f"Sin CV : L2={L2_base:.2e}  |∫ψ²-1|={norm_base:.2e}")
    print(f"Con CV : L2={mean_L2:.2e} ± {std_L2:.2e}  |∫ψ²-1|={mean_norm:.2e} ± {std_norm:.2e}")

    # Figuras comparativas
    os.makedirs(path("comp"), exist_ok=True)  # asegura carpeta comp

    x = np.arange(2)

    plt.figure(figsize=(5.4,4))
    plt.bar(x, [L2_base, mean_L2], yerr=[0.0, std_L2], tick_label=["Sin CV","Con CV"])
    plt.yscale("log"); plt.ylabel("Error L2 (log)")
    plt.title(f"Comparación L2 (n={n})"); plt.tight_layout()
    plt.savefig(path("comp", f"comp_L2_n{n}.png"), dpi=160); plt.close()

    plt.figure(figsize=(5.4,4))
    plt.bar(x, [norm_base, mean_norm], yerr=[0.0, std_norm], tick_label=["Sin CV","Con CV"])
    plt.yscale("log"); plt.ylabel(r"|∫ψ²-1| (log)")
    plt.title(f"Comparación normalización (n={n})"); plt.tight_layout()
    plt.savefig(path("comp", f"comp_norm_n{n}.png"), dpi=160); plt.close()

    return {
        "n": n,
        "L2_sinCV": float(L2_base),
        "L2_conCV_mean": float(mean_L2),
        "L2_conCV_std": float(std_L2),
        "NormErr_sinCV": float(norm_base),
        "NormErr_conCV_mean": float(mean_norm),
        "NormErr_conCV_std": float(std_norm),
    }

def compare_sweep(n_max=6, k_folds=5, seed=0):
    rows = []
    for n in range(1, n_max+1):
        rows.append(compare_single_vs_cv(n, k_folds=k_folds, seed=seed))

    # CSV resumen para la tesis
    with open(path("comp", "resumen_comp.csv"), "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["n","L2_sinCV","L2_conCV_mean","L2_conCV_std",
                    "NormErr_sinCV","NormErr_conCV_mean","NormErr_conCV_std"])
        for r in rows:
            w.writerow([r['n'], r['L2_sinCV'], r['L2_conCV_mean'], r['L2_conCV_std'],
                        r['NormErr_sinCV'], r['NormErr_conCV_mean'], r['NormErr_conCV_std']])

    # Figura global L2 vs n
    ns = [r["n"] for r in rows]
    L2_single = [r["L2_sinCV"] for r in rows]
    L2_mean   = [r["L2_conCV_mean"] for r in rows]
    L2_std    = [r["L2_conCV_std"] for r in rows]

    plt.figure(figsize=(7,4.5))
    plt.plot(ns, L2_single, "s--", label="Sin CV")
    plt.errorbar(ns, L2_mean, yerr=L2_std, fmt="o-", label="Con CV (media ± σ)")
    plt.yscale("log"); plt.xlabel("Modo n"); plt.ylabel("Error L2 (log)")
    plt.title("PINN: comparación con vs sin validación cruzada")
    plt.tight_layout()
    plt.savefig(path("comp", "comp_global_L2.png"), dpi=180); plt.close()

    print("Archivos en", path("comp"))
    print(" - resumen_comp.csv")
    print(" - comp_L2_n*.png, comp_norm_n*.png")
    print(" - comp_global_L2.png")
    return rows

# ===================== LANZADOR =====================
if __name__ == "__main__":
    # Ejemplo: barrido n = 1..6 y CV con 5 folds
    rows = compare_sweep(n_max=6, k_folds=5, seed=0)

    # Si quieres sólo entrenar sin CV todos los modos, descomenta:
    # all_results = []
    # for n in range(1, 7):
    #     res = run_one_mode(n, save_subdir=f"solo_sin_cv/mode_{n}")
    #     print(f"[OK] n={n}  L2={res['L2']:.2e}  ∫|ψ|²≈{res['integral']:.3f}  fig={res['fig_path']}")
    #     all_results.append(res)
    # # figura apilada
    # plt.figure(figsize=(10,7))
    # offset = 2.2
    # for r in all_results:
    #     y0 = offset*(r['n']-1)
    #     plt.plot(r['xs'], r['psi_exact'] + y0, "--", color="gray", linewidth=1)
    #     plt.plot(r['xs'], r['psi_pred'] + y0, label=f"ψ_{r['n']}")
    # plt.xlabel("x"); plt.ylabel("ψ_n(x) (desplazadas)")
    # plt.title("Pozo infinito — PINN vs exactas")
    # plt.legend(ncol=3)
    # plt.savefig(path("solo_sin_cv", "modos_apilados.png"), dpi=160, bbox_inches="tight")
    # plt.close()


2025-10-08 12:13:44.749117: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-08 12:13:45.163064: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-08 12:13:46.697627: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Política actual: <DTypePolicy "mixed_float16">


I0000 00:00:1759943627.468956    9145 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5561 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9
2025-10-08 12:13:48.299712: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


n=1 ep 1 | LPDE= 0.0211397056  Lnorm= 0.999828696  L= 40.0142899  ∫|ψ|²≈ 8.56604893e-05  λ= 40.0
n=1 ep 800 | LPDE= 0.00193613791  Lnorm= 6.19024831e-11  L= 0.00193614035  ∫|ψ|²≈ 1.00000787  λ= 40.0
n=1 ep 1600 | LPDE= 0.00130174553  Lnorm= 3.94430231e-08  L= 0.00130214  ∫|ψ|²≈ 0.999801397  λ= 10.0
n=1 ep 2400 | LPDE= 0.000732463785  Lnorm= 2.29206876e-10  L= 0.000732466055  ∫|ψ|²≈ 0.99998486  λ= 10.0
n=1 ep 3200 | LPDE= 0.00036306621  Lnorm= 2.89379756e-08  L= 0.00036335559  ∫|ψ|²≈ 1.00017011  λ= 10.0
n=1 ep 4000 | LPDE= 0.000191078696  Lnorm= 5.57122348e-10  L= 0.000191084269  ∫|ψ|²≈ 1.0000236  λ= 10.0

=== VALIDACIÓN CRUZADA para modo n=1 con 5 folds ===

n=1 ep 1 | LPDE= 0.0211397056  Lnorm= 0.999828696  L= 40.0142899  ∫|ψ|²≈ 8.56604893e-05  λ= 40.0
n=1 ep 800 | LPDE= 0.00193677412  Lnorm= 9.90816318e-11  L= 0.00193677808  ∫|ψ|²≈ 0.999990046  λ= 40.0
n=1 ep 1600 | LPDE= 0.00129236409  Lnorm= 4.4884132e-09  L= 0.00129240903  ∫|ψ|²≈ 0.999933  λ= 10.0
n=1 ep 2400 | LPDE= 0.00072820048